In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [49]:
df = pd.read_csv('train.csv')
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Cabin'] = df['Cabin'].fillna(df['Cabin'].mode())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode())
df['Fare'] = df['Fare'].fillna(df['Fare'].median())

In [50]:
df['Parch'] = np.where(df['Parch']==9, 6, df['Parch'])
df_dummies = pd.get_dummies(df, columns=['Sex', 'Pclass', 'SibSp', 'Parch', 'Embarked'])


In [51]:
scaler = StandardScaler()
df_dummies['Age_scale'] = scaler.fit_transform(df_dummies.loc[:, ['Age']])
df_dummies['Fare_scale'] = scaler.fit_transform(df_dummies.loc[:, ['Fare']])

In [52]:
c = 1.0
model = LogisticRegression(C=c, solver='liblinear', random_state=0)

In [53]:
df_dummies

,PassengerId,Survived,Name,Age,Ticket,Fare,Cabin,Sex_female,Sex_male,Pclass_1,...,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S,Age_scale,Fare_scale
0,1,0,"Braund, Mr. Owen Harris",22.0,A/5 21171,7.2500,B96 B98,False,True,False,...,False,False,False,False,False,False,False,True,-0.565736,-0.502445
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,PC 17599,71.2833,C85,True,False,True,...,False,False,False,False,False,True,False,False,0.663861,0.786845
2,3,1,"Heikkinen, Miss. Laina",26.0,STON/O2. 3101282,7.9250,G6,True,False,False,...,False,False,False,False,False,False,False,True,-0.258337,-0.488854
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,113803,53.1000,C123,True,False,True,...,False,False,False,False,False,False,False,True,0.433312,0.420730
4,5,0,"Allen, Mr. William Henry",35.0,373450,8.0500,NaN,False,True,False,...,False,False,False,False,False,False,False,True,0.433312,-0.486337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,"Montvila, Rev. Juozas",27.0,211536,13.0000,NaN,False,True,False,...,False,False,False,False,False,False,False,True,-0.181487,-0.386671
887,888,1,"Graham, Miss. Margaret Edith",19.0,112053,30.0000,B42,True,False,True,...,False,False,False,False,False,False,False,True,-0.796286,-0.044381
888,889,0,"Johnston, Miss. Catherine Helen ""Carrie""",28.0,W./C. 6607,23.4500,NaN,True,False,False,...,True,False,False,False,False,False,False,True,-0.104637,-0.176263
889,890,1,"Behr, Mr. Karl Howell",26.0,111369,30.0000,C148,False,True,True,...,False,False,False,False,False,True,False,False,-0.258337,-0.044381


In [54]:
x = df_dummies.drop(['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
y = df_dummies['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
model.fit(x_train, y_train)
model.score(x_test, y_test)


0.8059701492537313

In [59]:
test_data = pd.read_csv('test.csv')
test_data['Parch'] = np.where(test_data['Parch']==9, 6, test_data['Parch'])
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())
test_data['Cabin'] = test_data['Cabin'].fillna(test_data['Cabin'].mode())
test_data['Embarked'] = test_data['Embarked'].fillna(test_data['Embarked'].mode())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())
test_data_dummies = pd.get_dummies(test_data, columns=['Sex', 'Pclass', 'SibSp', 'Parch', 'Embarked'])
scaler = StandardScaler()
test_data_dummies['Age_scale'] = scaler.fit_transform(test_data_dummies.loc[:, ['Age']])
test_data_dummies['Fare_scale'] = scaler.fit_transform(test_data_dummies.loc[:, ['Fare']])

In [62]:
test_x = test_data_dummies.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
predictions = model.predict(test_x)
output = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': predictions
})
output.to_csv('submission.csv', index=False)